In [2]:
import matplotlib.pyplot as plt
import numpy as np

def promedio_varianza(x_data, y_data, num_clases, class_names):
    
    class_averages = np.zeros((num_clases, 28, 28), dtype=np.float32)
    class_vars = np.zeros((num_clases, 28, 28), dtype=np.float32)
    class_counts = np.zeros(num_clases, dtype=np.int32)

    # Media
    for image, label in zip(x_data, y_data):
        class_averages[label] += image
        class_counts[label] += 1

    for i in range(num_clases):
        class_averages[i] /= class_counts[i]

    # Varianza
    for image, label in zip(x_data, y_data):
        class_vars[label] += (image - class_averages[label]) ** 2
        
    for i in range(num_clases):
        class_vars[i] /= class_counts[i]
        
    # Visualizar los promedios de las imágenes
    plt.figure(figsize=(10, 5))

    for i in range(num_clases):
        plt.subplot(5, 4, 2*i + 1)
        plt.imshow(class_averages[i], cmap='gray')
        plt.title('Promedio ' + class_names[i])
        plt.axis('off')
        
        plt.subplot(5, 4, 2*i + 2)
        plt.imshow(class_vars[i], cmap='gray')
        plt.title('Varianza ' + class_names[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [2]:
def histogramas_de_intensidades(imagenes_por_clase, num_clases, bins_num=64):
    
    histogramas_por_clase = []

    # Calcular los histogramas de intensidades para cada clase
    for imagenes in imagenes_por_clase:
        intensities = imagenes.reshape(-1)
        hist, bins = np.histogram(intensities, bins=bins_num, range=(0.01, 1.0))   # excluye 0 para obtener un mejor gráfico
        histograma_normalizado = hist / np.sum(hist)
        histogramas_por_clase.append(histograma_normalizado)

    # Ploteo de los histogramas de intensidades para cada clase
    fig, axs = plt.subplots(nrows=num_clases, ncols=1, figsize=(5, 15))
    for clase, histograma, ax in zip(class_names, histogramas_por_clase, axs):
        ax.bar(bins[:-1], histograma, width=1.0/(2*bins_num), align='edge')
        ax.set_title('Histograma de intensidades: ' + clase)
        ax.set_xlabel('Intensidad')
        ax.set_ylabel('Frecuencia')

    plt.tight_layout()
    plt.show()

In [ ]:
def histogramas_de_inclinaciones(tablas_por_clase, num_clases, bins_num=64):
    
    histogramas_por_clase = []

    # Calcular los histogramas de inclinaciones para cada clase
    for tabla in tablas_por_clase:
        inclinaciones = tabla['slant'].values
        hist, bins = np.histogram(inclinaciones, bins=bins_num)
        histograma_normalizado = hist / np.sum(hist)
        histogramas_por_clase.append(histograma_normalizado)

    # Ploteo de los histogramas de inclinaciones para cada clase
    fig, axs = plt.subplots(nrows=num_clases, ncols=1, figsize=(5, 15))
    for clase, histograma, ax in zip(class_names, histogramas_por_clase, axs):
        ax.bar(bins[:-1], histograma, width=1.0/(2*bins_num), align='edge')
        ax.set_title('Histograma de inclinaciones: ' + clase)
        ax.set_xlabel('Intensidad')
        ax.set_ylabel('Frecuencia')

    plt.tight_layout()
    plt.show()

In [2]:
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator

def data_augmentation(x_data, y_data, horizontal_flip=False):    
    
    # Flippea horizontalmente todas las imágenes si HORIZONTAL_FLIP = True
    if horizontal_flip:
      flip_images = np.flip(x_data, axis=2)
      augmented_images_aux = np.concatenate((x_data, flip_images), axis=0)
      augmented_labels_aux = np.concatenate((y_data, y_data), axis=0)
    else:
      augmented_images_aux = x_data
      augmented_labels_aux = y_data

    # Crea un generador de data augmentation
    datagen = ImageDataGenerator(
            rotation_range=0,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1)

    # Ajusta el generador de datos al conjunto de entrenamiento
    datagen.fit(augmented_images_aux.reshape(-1, 28, 28, 1))

    # Genera nuevos datos de entrenamiento utilizando el generador de datos
    augmented_images = datagen.flow(augmented_images_aux.reshape(-1, 28, 28, 1), augmented_labels_aux, batch_size=240000)

    # Concatenar los nuevos datos de entrenamiento con los datos originales
    x_data_augmented = np.concatenate((augmented_images_aux, augmented_images[0][0].reshape(-1, 28, 28)), axis=0)
    y_data_augmented = np.concatenate((augmented_labels_aux, augmented_images[0][1]), axis=0)

    # Separa los datos aumentados por categoría
    imagenes_por_clase_augmented = [x_data_augmented[y_data_augmented == clase] for clase in clases]  # 10 arreglos de 12k matrices de 28x28
    
    return x_data_augmented, y_data_augmented, imagenes_por_clase_augmented

In [2]:
def plot_data_augmented(imagenes_por_clase_augmented, num_clases):
    
    cantidad_de_imagenes_por_clase = 20
    cantidad_de_imagenes_por_fila = 10
    filas = cantidad_de_imagenes_por_clase // cantidad_de_imagenes_por_fila

    fig, axs = plt.subplots(nrows=num_clases*filas, ncols=cantidad_de_imagenes_por_fila, figsize=(10, 25))
    for i, (clase, imagenes) in enumerate(zip(clases, imagenes_por_clase_augmented)):
        indices_aleatorios = np.random.choice(len(imagenes), size=cantidad_de_imagenes_por_clase, replace=False)
        imagenes_seleccionadas = imagenes[indices_aleatorios]
        imagenes_en_filas = np.split(imagenes_seleccionadas, filas)
        axs[i*filas, 0].set_title(class_names[clase])
        for j, imagenes_en_fila in enumerate(imagenes_en_filas):
            for k, imagen in enumerate(imagenes_en_fila):
                axs[i*filas+j, k].imshow(imagen, cmap='gray')
                axs[i*filas+j, k].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
from tensorflow import keras
from keras.layers import Flatten, Dense, Dropout
from keras_tuner.tuners import RandomSearch
from keras.optimizers import Adam

# Define la función para construir el modelo en el tuner
# Tengo que respetar que siempre: Flatten --> (Dense --> Dropout) --> (...) --> Out
def build_model_tuner(hp):
    
    model = keras.Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    for i in range(hp.Int('layers', 2, 3)):
        neurons = [x / (2**i) for x in [128, 256, 512, 1024]]
        model.add(Dense(units=hp.Choice(f'unit_{i+1}', values=neurons), activation='relu',
                        kernel_initializer=hp.Choice(f'initializer_{i+1}', values=['glorot_uniform', 'glorot_normal'])))
        model.add(Dropout(hp.Float(f'dropout_{i+1}', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(10, activation='softmax'))

    batch_size = hp.Choice('batch_size', values=[32, 64, 128, 256])
    
    model.compile(optimizer=Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                      
    return model

In [1]:
import matplotlib.pyplot as plt

def plot_tuner_results(accuracy):    
    
    fig, ax = plt.subplots()

    # Crear la figura y los subplots
    ax.plot([(n+1) for n in range(len(accuracy))], accuracy, marker='o')#, figsize=(7, 10))

    #ax.plot([(n+1) for n in range(len(accuracy))], accuracy, marker='o')
    ax.legend('Accuracy')
    ax.set_title('Accuracy vs Models')
    ax.set_xlabel('Model')
    ax.set_ylabel('Accuracy')
    ax.grid(True)
    
    #ax[1].plot([(n+1) for n in range(len(accuracy))], loss, marker='o')
    #ax[1].legend('Loss')
    #ax[1].set_title('Loss vs Models')
    #ax[1].set_xlabel('Model')
    #ax[1].set_ylabel('Loss')
    #ax[1].grid(True)
    
    # Ajustar los subplots
    plt.tight_layout()

    # Mostrar la figura
    plt.show()

In [2]:
# Tengo que respetar que siempre: Flatten --> Dense --> Dropout --> ... --> Out
def create_model(act, init, drop, norm):

    model = keras.Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    if norm[0] == True:
        model.add(BatchNormalization())
    model.add(Dense(ref_model.get_layer(1), activation='relu', kernel_initializer=init))
    model.add(Dropout(drop))
    if norm[1] == True:
        model.add(BatchNormalization())
    model.add(Dense(256, activation=act, kernel_initializer=init))
    model.add(Dropout(drop))
    if norm[2] == True:
        model.add(BatchNormalization())
    model.add(Dense(10, activation='softmax', kernel_initializer=init))
    if norm[3] == True:
        model.add(BatchNormalization())
    
    return model

In [ ]:
def compile_model(model, opt):
    
    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
def train_model(x_train, y_train, x_val, y_val, model, batches, callbacks):
    
    history = model.fit(x = x_train,
              y = y_train,
              batch_size = batches,
              epochs = 200,
              verbose = 1,
              validation_data = (x_val, y_val),
              shuffle = False,
              callbacks=callbacks)
    
    return model, history

In [ ]:
def plot_curves(variables, accuracies, epochs, variable_name, xscale):
    
    # Crear la figura y los subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    
    # Graficar en el primer subplot
    ax1.plot(variables, [max(accuracy) for accuracy in accuracies])
    ax1.set_title('Accuracy vs ' + variable_name)
    ax1.set_xscale(xscale)
    ax1.set_xlabel(variable_name)
    ax1.set_ylabel('Accuracy')
    ax1.grid(True)

    # Graficar en el segundo subplot
    ax2.plot(variables, [max(epoch) for epoch in epochs])
    ax2.set_title('Epochs vs ' + variable_name)
    ax2.set_xscale(xscale)
    ax2.set_xlabel(variable_name)
    ax2.set_ylabel('Epochs')
    ax2.grid(True)

    # Ajustar los subplots
    plt.tight_layout()

    # Mostrar la figura
    plt.show()

In [ ]:
def plot_bars(categories, accuracies, epochs, variable_name):
    
    # Configuración del tamaño de la figura
    plt.figure(figsize=(10, 5))
    cmap = plt.get_cmap('tab10')
    colors = [cmap(i) for i in np.linspace(0, 1, len(categories))]
    
    # Crear el primer gráfico de barras
    max_accs = [max(accuracy) for accuracy in accuracies]
    plt.subplot(1, 2, 1)
    bars = plt.bar(categories, max_accs, color=colors)
    plt.title('Accuracy para ' + variable_name)
    plt.xlabel(variable_name)
    plt.xscale('linear')
    plt.ylabel('Accuracy')
    plt.ylim(np.min(max_accs) - 2e-3, np.max(max_accs) + 2e-3)
    plt.legend(bars, categories)
    #plt.colorbar(orientation='vertical').set_label('Legend')

    # Crear el segundo gráfico de barras
    max_epochs = [max(epoch) for epoch in epochs]
    plt.subplot(1, 2, 2)
    bars = plt.bar(categories, max_epochs, color=colors)
    plt.title('Epochs para ' + variable_name)
    plt.xlabel(variable_name)
    plt.xscale('linear')
    plt.ylabel('Epochs')
    plt.ylim(np.min(max_epochs) - 5, np.max(max_epochs) + 5)
    plt.legend(bars, categories)
    #plt.colorbar(orientation='vertical').set_label('Legend')

    # Ajustar los subplots
    plt.tight_layout()

    # Mostrar los gráficos
    plt.show()

In [2]:
def plot_acc_vs_epochs(accuracies, epochs, variable_labels):
    
    min_epochs = min([len(epoch) for epoch in epochs])
    for i in range(len(variable_labels)):
        plt.plot(epochs[i][:min_epochs], accuracies[i][:min_epochs], label=variable_labels[i])
        
    plt.title('Accuracy vs Epochs')
    plt.xscale('linear')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
        
    # Ajustar los subplots
    plt.tight_layout()

    # Mostrar los gráficos
    plt.show()